In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'



In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np


2023-07-13 06:06:55.492479: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
tf.__version__

'2.5.0'

In [4]:
from tensorflow.keras import backend
backend.set_image_data_format('channels_last')

In [5]:
# # models.py
# import tensorflow as tf
# from tensorflow.keras import Model
# from tensorflow.keras.applications import MobileNetV2, ResNet50
# from tensorflow.keras.layers import Input, Conv2D, ReLU, LeakyReLU
# from modules.anchor import decode_tf, prior_box_tf


# def _regularizer(weights_decay):
#     """l2 regularizer"""
#     return tf.keras.regularizers.l2(weights_decay)


# def _kernel_init(scale=1.0, seed=None):
#     """He normal initializer"""
#     return tf.keras.initializers.he_normal()


# class BatchNormalization(tf.keras.layers.BatchNormalization):
#     """Make trainable=False freeze BN for real (the og version is sad).
#        ref: https://github.com/zzh8829/yolov3-tf2
#     """
#     def __init__(self, axis=-1, momentum=0.9, epsilon=1e-5, center=True,
#                  scale=True, name=None, **kwargs):
#         super(BatchNormalization, self).__init__(
#             axis=axis, momentum=momentum, epsilon=epsilon, center=center,
#             scale=scale, name=name, **kwargs)

#     def call(self, x, training=False):
#         if training is None:
#             training = tf.constant(False)
#         training = tf.logical_and(training, self.trainable)

#         return super().call(x, training)


# def Backbone(backbone_type='ResNet50', use_pretrain=True):
#     """Backbone Model"""
#     weights = None
#     if use_pretrain:
#         weights = 'imagenet'

#     def backbone(x):
#         if backbone_type == 'ResNet50':
#             extractor = ResNet50(
#                 input_shape=x.shape[1:], include_top=False, weights=weights)
#             pick_layer1 = 80  # [80, 80, 512]
#             pick_layer2 = 142  # [40, 40, 1024]
#             pick_layer3 = 174  # [20, 20, 2048]
#             preprocess = tf.keras.applications.resnet.preprocess_input
#         elif backbone_type == 'MobileNetV2':
#             extractor = MobileNetV2(
#                 input_shape=x.shape[1:], include_top=False, weights=weights)
#             pick_layer1 = 54  # [80, 80, 32]
#             pick_layer2 = 116  # [40, 40, 96]
#             pick_layer3 = 143  # [20, 20, 160]
#             preprocess = tf.keras.applications.mobilenet_v2.preprocess_input
#         else:
#             raise NotImplementedError(
#                 'Backbone type {} is not recognized.'.format(backbone_type))

#         return Model(extractor.input,
#                      (extractor.layers[pick_layer1].output,
#                       extractor.layers[pick_layer2].output,
#                       extractor.layers[pick_layer3].output),
#                      name=backbone_type + '_extrator')(preprocess(x))

#     return backbone


# class ConvUnit(tf.keras.layers.Layer):
#     """Conv + BN + Act"""
#     def __init__(self, f, k, s, wd, act=None, name='ConvBN', **kwargs):
#         super(ConvUnit, self).__init__(name=name, **kwargs)
#         self.conv = Conv2D(filters=f, kernel_size=k, strides=s, padding='same',
#                            kernel_initializer=_kernel_init(),
#                            kernel_regularizer=_regularizer(wd),
#                            use_bias=False, name='conv')
#         self.bn = BatchNormalization(name='bn')

#         if act is None:
#             self.act_fn = tf.identity
#         elif act == 'relu':
#             self.act_fn = ReLU()
#         elif act == 'lrelu':
#             self.act_fn = LeakyReLU(0.1)
#         else:
#             raise NotImplementedError(
#                 'Activation function type {} is not recognized.'.format(act))

#     def call(self, x):
#         return self.act_fn(self.bn(self.conv(x)))


# class FPN(tf.keras.layers.Layer):
#     """Feature Pyramid Network"""

#     def __init__(self, out_ch, wd, name='FPN', **kwargs):
#         super(FPN, self).__init__(name=name, **kwargs)
#         act = 'relu'
#         if (out_ch <= 64):
#             act = 'lrelu'

#         self.output1 = ConvUnit(f=out_ch, k=1, s=1, wd=wd, act=act)
#         self.output2 = ConvUnit(f=out_ch, k=1, s=1, wd=wd, act=act)
#         self.output3 = ConvUnit(f=out_ch, k=1, s=1, wd=wd, act=act)
#         self.merge1 = ConvUnit(f=out_ch, k=3, s=1, wd=wd, act=act)
#         self.merge2 = ConvUnit(f=out_ch, k=3, s=1, wd=wd, act=act)

#     def call(self, x):
#         output1 = self.output1(x[0])  # [80, 80, out_ch]
#         output2 = self.output2(x[1])  # [40, 40, out_ch]
#         output3 = self.output3(x[2])  # [20, 20, out_ch]

#         up_h, up_w = tf.shape(output2)[1], tf.shape(output2)[2]
#         up3 = tf.image.resize(output3, [up_h, up_w], method='nearest')
#         output2 = output2 + up3
#         output2 = self.merge2(output2)

#         up_h, up_w = tf.shape(output1)[1], tf.shape(output1)[2]
#         up2 = tf.image.resize(output2, [up_h, up_w], method='nearest')
#         output1 = output1 + up2
#         output1 = self.merge1(output1)

#         return output1, output2, output3


# class SSH(tf.keras.layers.Layer):
#     """Single Stage Headless Layer"""
#     def __init__(self, out_ch, wd, name='SSH', **kwargs):
#         super(SSH, self).__init__(name=name, **kwargs)
#         assert out_ch % 4 == 0
#         act = 'relu'
#         if (out_ch <= 64):
#             act = 'lrelu'

#         self.conv_3x3 = ConvUnit(f=out_ch // 2, k=3, s=1, wd=wd, act=None)

#         self.conv_5x5_1 = ConvUnit(f=out_ch // 4, k=3, s=1, wd=wd, act=act)
#         self.conv_5x5_2 = ConvUnit(f=out_ch // 4, k=3, s=1, wd=wd, act=None)

#         self.conv_7x7_2 = ConvUnit(f=out_ch // 4, k=3, s=1, wd=wd, act=act)
#         self.conv_7x7_3 = ConvUnit(f=out_ch // 4, k=3, s=1, wd=wd, act=None)

#         self.relu = ReLU()

#     def call(self, x):
#         conv_3x3 = self.conv_3x3(x)

#         conv_5x5_1 = self.conv_5x5_1(x)
#         conv_5x5 = self.conv_5x5_2(conv_5x5_1)

#         conv_7x7_2 = self.conv_7x7_2(conv_5x5_1)
#         conv_7x7 = self.conv_7x7_3(conv_7x7_2)

#         output = tf.concat([conv_3x3, conv_5x5, conv_7x7], axis=3)
#         output = self.relu(output)

#         return output


# class BboxHead(tf.keras.layers.Layer):
#     """Bbox Head Layer"""
#     def __init__(self, num_anchor, wd, name='BboxHead', **kwargs):
#         super(BboxHead, self).__init__(name=name, **kwargs)
#         self.num_anchor = num_anchor
#         self.conv = Conv2D(filters=num_anchor * 4, kernel_size=1, strides=1)

#     def call(self, x):
#         h, w = tf.shape(x)[1], tf.shape(x)[2]
#         x = self.conv(x)

#         return tf.reshape(x, [-1, h * w * self.num_anchor, 4])


# class LandmarkHead(tf.keras.layers.Layer):
#     """Landmark Head Layer"""
#     def __init__(self, num_anchor, wd, name='LandmarkHead', **kwargs):
#         super(LandmarkHead, self).__init__(name=name, **kwargs)
#         self.num_anchor = num_anchor
#         self.conv = Conv2D(filters=num_anchor * 10, kernel_size=1, strides=1)

#     def call(self, x):
#         h, w = tf.shape(x)[1], tf.shape(x)[2]
#         x = self.conv(x)

#         return tf.reshape(x, [-1, h * w * self.num_anchor, 10])


# class ClassHead(tf.keras.layers.Layer):
#     """Class Head Layer"""
#     def __init__(self, num_anchor, wd, name='ClassHead', **kwargs):
#         super(ClassHead, self).__init__(name=name, **kwargs)
#         self.num_anchor = num_anchor
#         self.conv = Conv2D(filters=num_anchor * 2, kernel_size=1, strides=1)

#     def call(self, x):
#         h, w = tf.shape(x)[1], tf.shape(x)[2]
#         x = self.conv(x)

#         # print('*'*100)
#         # print(tf.shape(x))
#         return tf.reshape(x, [-1, h * w * self.num_anchor, 2])


# def RetinaFaceModel(cfg, training=False, iou_th=0.4, score_th=0.02,
#                     name='RetinaFaceModel'):
#     """Retina Face Model"""
#     tf.keras.backend.set_image_data_format('channels_last')
#     input_size = cfg['input_size'] if training else None
#     wd = cfg['weights_decay']
#     out_ch = cfg['out_channel']
#     num_anchor = len(cfg['min_sizes'][0])
#     backbone_type = cfg['backbone_type']

#     # define model
#     x = inputs = Input([input_size, input_size, 3], name='input_image')

#     x = Backbone(backbone_type=backbone_type)(x)

#     fpn = FPN(out_ch=out_ch, wd=wd)(x)
    
#     out = fpn
# #     features = [SSH(out_ch=out_ch, wd=wd, name=f'SSH_{i}')(f)
# #                 for i, f in enumerate(fpn)]

# #     bbox_regressions = tf.concat(
# #         [BboxHead(num_anchor, wd=wd, name=f'BboxHead_{i}')(f)
# #          for i, f in enumerate(features)], axis=1)
# #     # landm_regressions = tf.concat(
# #     #     [LandmarkHead(num_anchor, wd=wd, name=f'LandmarkHead_{i}')(f)
# #     #      for i, f in enumerate(features)], axis=1)
# #     classifications = tf.concat(
# #         [ClassHead(num_anchor, wd=wd, name=f'ClassHead_{i}')(f)
# #          for i, f in enumerate(features)], axis=1)

# #     classifications = tf.keras.layers.Softmax(axis=-1)(classifications)

# #     if training:
# #         out = (bbox_regressions, 
# #                # landm_regressions, 
# #                classifications)
# #     else:
# #         out = (bbox_regressions, 
# #                # landm_regressions, 
# #                classifications)


# #         # only for batch size 1
# #         preds = tf.concat(  # [bboxes, landms, landms_valid, conf]
# #             [bbox_regressions[0], 
# #              # landm_regressions[0],
# #              # tf.ones_like(classifications[0, :, 0][..., tf.newaxis]),
# #              classifications[0, :, 1][..., tf.newaxis]
# #             ], 1)
# #         priors = prior_box_tf((tf.shape(inputs)[1], tf.shape(inputs)[2]),
# #                               cfg['min_sizes'],  cfg['steps'], cfg['clip'])
# #         decode_preds = decode_tf(preds, priors, cfg['variances'])

# #         selected_indices = tf.image.non_max_suppression(
# #             boxes=decode_preds[:, :4],
# #             scores=decode_preds[:, -1],
# #             max_output_size=tf.shape(decode_preds)[0],
# #             iou_threshold=iou_th,
# #             score_threshold=score_th)

# #         out = tf.gather(decode_preds, selected_indices)

#     return Model(inputs, out, name=name)


In [6]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.layers import Input, Conv2D, ReLU, LeakyReLU
from modules.anchor import decode_tf, prior_box_tf


def _regularizer(weights_decay):
    """l2 regularizer"""
    return tf.keras.regularizers.l2(weights_decay)


def _kernel_init(scale=1.0, seed=None):
    """He normal initializer"""
    return tf.keras.initializers.he_normal()


class BatchNormalization(tf.keras.layers.BatchNormalization):
    """Make trainable=False freeze BN for real (the og version is sad).
       ref: https://github.com/zzh8829/yolov3-tf2
    """
    def __init__(self, axis=-1, momentum=0.9, epsilon=1e-5, center=True,
                 scale=True, name=None, **kwargs):
        super(BatchNormalization, self).__init__(
            axis=axis, momentum=momentum, epsilon=epsilon, center=center,
            scale=scale, name=name, **kwargs)

    def call(self, x, training=False):
        if training is None:
            training = tf.constant(False)
        training = tf.logical_and(training, self.trainable)

        return super().call(x, training)


def Backbone(backbone_type='ResNet50', use_pretrain=True):
    """Backbone Model"""
    weights = None
    if use_pretrain:
        weights = 'imagenet'

    def backbone(x):
        if backbone_type == 'ResNet50':
            extractor = ResNet50(
                input_shape=x.shape[1:], include_top=False, weights=weights)
            pick_layer1 = 80  # [80, 80, 512]
            pick_layer2 = 142  # [40, 40, 1024]
            pick_layer3 = 174  # [20, 20, 2048]
            preprocess = tf.keras.applications.resnet.preprocess_input
        elif backbone_type == 'MobileNetV2':
            extractor = MobileNetV2(
                input_shape=x.shape[1:], include_top=False, weights=weights)
            pick_layer1 = 54  # [80, 80, 32]
            pick_layer2 = 116  # [40, 40, 96]
            pick_layer3 = 143  # [20, 20, 160]
            preprocess = tf.keras.applications.mobilenet_v2.preprocess_input
        else:
            raise NotImplementedError(
                'Backbone type {} is not recognized.'.format(backbone_type))

        return Model(extractor.input,
                     (extractor.layers[pick_layer1].output,
                      extractor.layers[pick_layer2].output,
                      extractor.layers[pick_layer3].output),
                     name=backbone_type + '_extrator')(preprocess(x))

    return backbone


class ConvUnit(tf.keras.layers.Layer):
    """Conv + BN + Act"""
    def __init__(self, f=None, k=None, s=None, wd=None, act=None, name='ConvBN', **kwargs):
        super(ConvUnit, self).__init__(name=name, **kwargs)
        self.f = f
        self.k = k
        self.s = s
        self.wd = wd
        self.act = act
#         self.name = name
        self.conv = Conv2D(filters=f, kernel_size=k, strides=s, padding='same',
                           kernel_initializer=_kernel_init(),
                           kernel_regularizer=_regularizer(wd),
                           use_bias=False, name='conv')

        # self.bn = BatchNormalization(name='bn')
        self.bn = tf.keras.layers.BatchNormalization(name='bn')

        if act is None:
            self.act_fn = tf.identity
        elif act == 'relu':
            self.act_fn = ReLU()
        elif act == 'lrelu':
            self.act_fn = LeakyReLU(0.1)
        else:
            raise NotImplementedError(
                'Activation function type {} is not recognized.'.format(act))


    def call(self, x):
        return self.act_fn(self.bn(self.conv(x)))
        # return self.bn(self.conv(x))
        # return self.act_fn(self.conv(x))

    def get_config(self):
        config = super().get_config()
        temp = {
            'f': self.f,
            'k': self.k,
            's': self.s,
            'wd': self.wd,
            'act': self.act,
#             'convunit_name': self.name,
#             'conv': self.conv,
#             'bn': self.bn,
#             'act_fn': self.act_fn,
        }
        config.update(temp)
        return config


class FPN(tf.keras.layers.Layer):
    """Feature Pyramid Network"""
    def __init__(self, out_ch, wd, name='FPN', **kwargs):
        super(FPN, self).__init__(name=name, **kwargs)
        self.act = 'relu'
        self.out_ch = out_ch
#         self.name = name
        self.wd = wd
        if (out_ch <= 64):
            self.act = 'lrelu'

        self.output1 = ConvUnit(f=out_ch, k=1, s=1, wd=wd, act=self.act, name=name+str(1))
        self.output2 = ConvUnit(f=out_ch, k=1, s=1, wd=wd, act=self.act, name=name+str(2))
        self.output3 = ConvUnit(f=out_ch, k=1, s=1, wd=wd, act=self.act, name=name+str(3))
        self.merge1 = ConvUnit(f=out_ch, k=3, s=1, wd=wd, act=self.act, name=name+str(4))
        self.merge2 = ConvUnit(f=out_ch, k=3, s=1, wd=wd, act=self.act, name=name+str(5))

    def call(self, x):
        print(f"x[0] :- {x[0].shape}")
        print(f"x[1] :- {x[1].shape}")
        print(f"x[2] :- {x[2].shape}")
        
        print(f"Running output1 conv layer")
        output1 = self.output1(x[0])  # [80, 80, out_ch]
        print(f"Output of output1 conv :- {output1.shape} \n")
        
        print(f"Running output2 conv layer")
        output2 = self.output2(x[1])  # [40, 40, out_ch]
        print(f"Output of output2 conv :- {output2.shape} \n")
        
        print(f"Running output3 conv layer")
        output3 = self.output3(x[2])  # [20, 20, out_ch]
        print(f"Output of output3 conv :- {output3.shape} \n")

        print(f"Running resize of output3 \n")
        up_h, up_w = tf.shape(output2)[1], tf.shape(output2)[2]
        
        # up_h, up_w = output2.shape[1], output2.shape[2]
        print(output2.shape[1], output2.shape[2])
        
        up3 = tf.image.resize(output3, [up_h, up_w], method='nearest')
        print(f"Resized output3 to :- {tf.shape(up3)} {up3.shape}\n")
        
        print(f"Adding output2 to resized output3 \n")
        # print(f"Shape of tf.concat output2 ,up3 :- {tf.add(output2, up3).shape}")
        
        # output2 = output2 + up3
        output2 = tf.add(output2, up3)
        print(f"Output2 shape :- {tf.shape(output2)} {output2.shape}\n")
        
        print(f"passing output2 to merge2 conv \n")
        output2 = self.merge2(output2)
        print(f"Output of output2 :- {output2.shape} \n")
        
        print(f"Running resize of output2 w.r.t. output1 \n")
        up_h, up_w = tf.shape(output1)[1], tf.shape(output1)[2]
        up2 = tf.image.resize(output2, [up_h, up_w], method='nearest')
        print(f"Resized output2 to :- {tf.shape(up2)} {up2.shape}\n")
        
        print(f"Adding output1 to resized output2 \n")
        
        # output1 = output1 + up2
        output1 = tf.add(output1, up2)
        print(f"Output1 shape :- {tf.shape(output1)} {output1.shape}\n")
        
        print(f"Running merge1 conv on output1 \n")
        output1 = self.merge1(output1)
        print(f"Shape of output1 :- {output1.shape} \n")

        return output1, output2, output3

    def get_config(self):
        config = super().get_config()
        temp = {
#             'fpn_act': self.act,
            'out_ch': self.out_ch,
#             'fpn_name': self.name,
            'wd': self.wd,
#             'fpn_conv1': self.output1,
#             'fpn_conv2': self.output2,
#             'fpn_conv3': self.output3,
#             'fpn_merge1': self.merge1,
#             'fpn_merge2': self.merge2,
        }
        config.update(temp)
        return config


class SSH(tf.keras.layers.Layer):
    """Single Stage Headless Layer"""
    def __init__(self, out_ch, wd, name='SSH', **kwargs):
        super(SSH, self).__init__(name=name, **kwargs)
        assert out_ch % 4 == 0
        self.out_ch = out_ch
#         self.name = name
        self.wd = wd
        self.act = 'relu'
        if (out_ch <= 64):
            self.act = 'lrelu'

        self.conv_3x3 = ConvUnit(f=out_ch // 2, k=3, s=1, wd=wd, act=None, name=name+str(1))

        self.conv_5x5_1 = ConvUnit(f=out_ch // 4, k=3, s=1, wd=wd, act=self.act, name=name+str(2))
        self.conv_5x5_2 = ConvUnit(f=out_ch // 4, k=3, s=1, wd=wd, act=None, name=name+str(3))

        self.conv_7x7_2 = ConvUnit(f=out_ch // 4, k=3, s=1, wd=wd, act=self.act, name=name+str(4))
        self.conv_7x7_3 = ConvUnit(f=out_ch // 4, k=3, s=1, wd=wd, act=None, name=name+str(5))

        self.relu = ReLU()

    def call(self, x):
        conv_3x3 = self.conv_3x3(x)

        conv_5x5_1 = self.conv_5x5_1(x)
        conv_5x5 = self.conv_5x5_2(conv_5x5_1)

        conv_7x7_2 = self.conv_7x7_2(conv_5x5_1)
        conv_7x7 = self.conv_7x7_3(conv_7x7_2)

        output = tf.concat([conv_3x3, conv_5x5, conv_7x7], axis=3)
        output = self.relu(output)

        return output
    
    def get_config(self):
        config = super().get_config()
        temp = {
            'out_ch': self.out_ch,
#             'ssh_name': self.name,
            'wd': self.wd,
#             'ssh_act': self.act,
#             'ssh_conv_3x3': self.conv_3x3,
#             'ssh_conv_5x5_1': self.conv_5x5_1,
#             'ssh_conv_5x5_2': self.conv_5x5_2,
#             'ssh_conv_7x7_2': self.conv_7x7_2,
#             'ssh_conv_7x7_3': self.conv_7x7_3,
#             'ssh_relu': self.relu,
        }
        config.update(temp)
        return config


class BboxHead(tf.keras.layers.Layer):
    """Bbox Head Layer"""
    def __init__(self, num_anchor, wd, name='BboxHead', **kwargs):
        super(BboxHead, self).__init__(name=name, **kwargs)
#         self.name = name
        self.wd = wd
        self.num_anchor = num_anchor
        self.conv = Conv2D(filters=num_anchor * 4, kernel_size=1, strides=1, name=name)

    def call(self, x):
        h, w = tf.shape(x)[1], tf.shape(x)[2]
        x = self.conv(x)

        return tf.reshape(x, [-1, h * w * self.num_anchor, 4])
    
    def get_config(self):
        config = super().get_config()
        temp = {
#             'bboxhead_name': self.name,
            'wd': self.wd,
            'num_anchor': self.num_anchor,
#             'bboxhead_conv': self.conv,
        }
        config.update(temp)
        return config


class LandmarkHead(tf.keras.layers.Layer):
    """Landmark Head Layer"""
    def __init__(self, num_anchor, wd, name='LandmarkHead', **kwargs):
        super(LandmarkHead, self).__init__(name=name, **kwargs)
        self.num_anchor = num_anchor
#         self.name = name
        self.wd = wd
        self.conv = Conv2D(filters=num_anchor * 10, kernel_size=1, strides=1, name=name)

    def call(self, x):
        h, w = tf.shape(x)[1], tf.shape(x)[2]
        x = self.conv(x)

        return tf.reshape(x, [-1, h * w * self.num_anchor, 10])
    
    def get_config(self):
        config = super().get_config()
        temp = {
#             'landmarkhead_name': self.name,
            'wd': self.wd,
            'num_anchor': self.num_anchor,
#             'landmarkhead_conv': self.conv,
        }
        config.update(temp)
        return config


class ClassHead(tf.keras.layers.Layer):
    """Class Head Layer"""
    def __init__(self, num_anchor, wd, name='ClassHead', **kwargs):
        super(ClassHead, self).__init__(name=name, **kwargs)
#         self.name = name
        self.wd = wd
        self.num_anchor = num_anchor
        self.conv = Conv2D(filters=num_anchor * 2, kernel_size=1, strides=1, name=name)

    def call(self, x):
        h, w = tf.shape(x)[1], tf.shape(x)[2]
        x = self.conv(x)

        # print('*'*100)
        # print(tf.shape(x))
        return tf.reshape(x, [-1, h * w * self.num_anchor, 2])
    
    def get_config(self):
        config = super().get_config()
        temp = {
#             'classhead_name': self.name,
            'wd': self.wd,
            'num_anchor': self.num_anchor,
#             'classhead_conv': self.conv,
        }
        config.update(temp)
        return config


def RetinaFaceModel(cfg, training=False, iou_th=0.4, score_th=0.02,
                    name='RetinaFaceModel'):
    """Retina Face Model"""
    tf.keras.backend.set_image_data_format('channels_last')
    input_size = cfg['input_size'] if training else None
    wd = cfg['weights_decay']
    out_ch = cfg['out_channel']
    print(out_ch)
    
    num_anchor = len(cfg['min_sizes'][0])
    backbone_type = cfg['backbone_type']

    # define model
    x = inputs = Input([input_size, input_size, 3], name='input_image')

    x = Backbone(backbone_type=backbone_type)(x)

    fpn = FPN(out_ch=out_ch, wd=wd)(x)

    # out = fpn

    features = [SSH(out_ch=out_ch, wd=wd, name=f'SSH_{i}')(f)
                for i, f in enumerate(fpn)]

    bbox_regressions = tf.concat(
        [BboxHead(num_anchor, wd=wd, name=f'BboxHead_{i}')(f)
         for i, f in enumerate(features)], axis=1)
    
    
    
    # landm_regressions = tf.concat(
    #     [LandmarkHead(num_anchor, wd=wd, name=f'LandmarkHead_{i}')(f)
    #      for i, f in enumerate(features)], axis=1)
    classifications = tf.concat(
        [ClassHead(num_anchor, wd=wd, name=f'ClassHead_{i}')(f)
         for i, f in enumerate(features)], axis=1)

    classifications = tf.keras.layers.Softmax(axis=-1)(classifications)
    
    
    bbox_reg = [BboxHead(num_anchor, wd=wd, name=f'BboxHead_{i}')(f)
         for i, f in enumerate(features)]
    
    
    clf = [ClassHead(num_anchor, wd=wd, name=f'ClassHead_{i}')(f)
         for i, f in enumerate(features)]
    
    if training:
        out = (bbox_regressions, 
               # landm_regressions, 
               classifications)
        # out = (bbox_reg, clf)
    else:
        out = (bbox_regressions, 
               # landm_regressions, 
               classifications)
        # out = (bbox_reg, clf)

        
#         # only for batch size 1
#         preds = tf.concat(  # [bboxes, landms, landms_valid, conf]
#             [bbox_regressions[0], 
#              # landm_regressions[0],
#              # tf.ones_like(classifications[0, :, 0][..., tf.newaxis]),
#              classifications[0, :, 1][..., tf.newaxis]
#             ], 1)
#         priors = prior_box_tf((tf.shape(inputs)[1], tf.shape(inputs)[2]),
#                               cfg['min_sizes'],  cfg['steps'], cfg['clip'])
#         decode_preds = decode_tf(preds, priors, cfg['variances'])

#         selected_indices = tf.image.non_max_suppression(
#             boxes=decode_preds[:, :4],
#             scores=decode_preds[:, -1],
#             max_output_size=tf.shape(decode_preds)[0],
#             iou_threshold=iou_th,
#             score_threshold=score_th)

#         out = tf.gather(decode_preds, selected_indices)

    return Model(inputs, out, name=name)

In [7]:
# import tf2onnx
# import onnxruntime as rt
import tensorflow as tf
# from tensorflow.python.keras.models import load_model
import os
# from modules.models import RetinaFaceModel
from modules.utils import (set_memory_growth, load_yaml, load_dataset,
                           ProgressBar)

In [8]:
cfg_path = './configs/retinaface_mbv2.yaml'
cfg = load_yaml(cfg_path)

In [9]:


model = RetinaFaceModel(cfg, training=False)

64


2023-07-13 06:06:59.959436: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-07-13 06:06:59.984212: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-13 06:06:59.984243: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ip-172-16-165-11.ec2.internal
2023-07-13 06:06:59.984250: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ip-172-16-165-11.ec2.internal
2023-07-13 06:06:59.984334: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.85.12
2023-07-13 06:06:59.984356: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.85.12
2023-07-13 06:06:59.984362: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 525.85.12
2023-07-13 06:06:59.9

x[0] :- (None, None, None, 192)
x[1] :- (None, None, None, 576)
x[2] :- (None, None, None, 960)
Running output1 conv layer
Output of output1 conv :- (None, None, None, 64) 

Running output2 conv layer
Output of output2 conv :- (None, None, None, 64) 

Running output3 conv layer
Output of output3 conv :- (None, None, None, 64) 

Running resize of output3 

None None
Resized output3 to :- Tensor("FPN/Shape_2:0", shape=(4,), dtype=int32) (None, None, None, 64)

Adding output2 to resized output3 

Output2 shape :- Tensor("FPN/Shape_3:0", shape=(4,), dtype=int32) (None, None, None, 64)

passing output2 to merge2 conv 

Output of output2 :- (None, None, None, 64) 

Running resize of output2 w.r.t. output1 

Resized output2 to :- Tensor("FPN/Shape_6:0", shape=(4,), dtype=int32) (None, None, None, 64)

Adding output1 to resized output2 

Output1 shape :- Tensor("FPN/Shape_7:0", shape=(4,), dtype=int32) (None, None, None, 64)

Running merge1 conv on output1 

Shape of output1 :- (None, None, No

In [10]:

# model.predict(np.random.rand(1, 800, 800, 3))

In [11]:
model.summary()

Model: "RetinaFaceModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, None, None,  0                                            
__________________________________________________________________________________________________
tf.math.truediv (TFOpLambda)    (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
tf.math.subtract (TFOpLambda)   (None, None, None, 3 0           tf.math.truediv[0][0]            
__________________________________________________________________________________________________
MobileNetV2_extrator (Functiona ((None, None, None,  1518464     tf.math.subtract[0][0]           
____________________________________________________________________________________

In [12]:
# !pip install pydot

In [13]:
# !pip install graphviz

In [14]:
# tf.keras.utils.plot_model(
#     model,
#     show_shapes=False,
#     show_dtype=False,
#     show_layer_names=True,
#     expand_nested=False,
#     dpi=96
# )

In [15]:
# checkpoint_dir = '/home/ec2-user/SageMaker/gaurav/retinaface-tf2_unlabelled_data_without_landmarks/all_checkpoints/checkpoint_1000'
checkpoint_dir = './ScratchML-Models/Models/16may2023/face_detection/portrait/checkpoint_78/'
checkpoint_dir = './ScratchML-Models/Models/16may2023/face_detection/landscape/checkpoint_2030/'
checkpoint = tf.train.Checkpoint(model=model)

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [16]:
model.summary()

Model: "RetinaFaceModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, None, None,  0                                            
__________________________________________________________________________________________________
tf.math.truediv (TFOpLambda)    (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
tf.math.subtract (TFOpLambda)   (None, None, None, 3 0           tf.math.truediv[0][0]            
__________________________________________________________________________________________________
MobileNetV2_extrator (Functiona ((None, None, None,  1518464     tf.math.subtract[0][0]           
____________________________________________________________________________________

In [17]:
# # Convert Keras model to ConcreteFunction
# full_model = tf.function(lambda x: model(x))
# full_model = full_model.get_concrete_function(
#     tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

In [18]:
# # tf.compat.v1.global_variables_initializer().run()
# # tf.initializers.global_variables_initializer().run()

# frozen_func = convert_variables_to_constants_v2(full_model)
# gph = frozen_func.graph.as_graph_def()

In [19]:
# type(gph)

In [20]:
# gph_li = list(gph.node)

In [21]:
# gph_li[587]

In [22]:
import coremltools as ct

In [24]:
import coremltools as ct


# # import coremltools as ct
# pipeline = ct.PassPipeline()
# pipeline.remove_passes({"common::fuse_conv_batchnorm"})


print(ct.__version__)

input_name = model.inputs[0].name

height = 256
width = 256

input_shape = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
                    ct.RangeDim(lower_bound=height, upper_bound=-1),
                    ct.RangeDim(lower_bound=width, upper_bound=-1),
                    3))

# c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], pass_pipeline=pipeline, source='tensorflow')


input_shape = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
                    800,
                    800,
                    3))

input_shape = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
                    1024,
                    1024,
                    3))


c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], 
                     # pass_pipeline=pipeline,
                     source='tensorflow')


6.3.0
x[0] :- (None, None, None, 192)
x[1] :- (None, None, None, 576)
x[2] :- (None, None, None, 960)
Running output1 conv layer
Output of output1 conv :- (None, None, None, 64) 

Running output2 conv layer
Output of output2 conv :- (None, None, None, 64) 

Running output3 conv layer
Output of output3 conv :- (None, None, None, 64) 

Running resize of output3 

None None
Resized output3 to :- Tensor("RetinaFaceModel/FPN/Shape_2:0", shape=(4,), dtype=int32) (None, None, None, 64)

Adding output2 to resized output3 

Output2 shape :- Tensor("RetinaFaceModel/FPN/Shape_3:0", shape=(4,), dtype=int32) (None, None, None, 64)

passing output2 to merge2 conv 

Output of output2 :- (None, None, None, 64) 

Running resize of output2 w.r.t. output1 

Resized output2 to :- Tensor("RetinaFaceModel/FPN/Shape_6:0", shape=(4,), dtype=int32) (None, None, None, 64)

Adding output1 to resized output2 

Output1 shape :- Tensor("RetinaFaceModel/FPN/Shape_7:0", shape=(4,), dtype=int32) (None, None, None, 64)

2023-07-13 06:07:44.646814: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-07-13 06:07:44.646973: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-07-13 06:07:44.660879: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.009ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2023-07-13 06:07:45.481740: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-07-13 06:07:45.481880: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-07-13 06:07:45.597411: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 721 nodes (-353), 1464 ed

AttributeError: 'concat' object has no attribute 'y'

In [32]:
c_model.save("./retinaface-1024.mlmodel")


In [41]:
c_model.save("./retinaface-800.mlmodel")


In [ ]:
from coremltools.models.neural_network import flexible_shape_utils

spec = ct.utils.load_spec('./retinaface-800.mlmodel')

# to update the shapes of the first input
input_name = spec.description.input[0].name


# Range shapes and multi array inputs
# -1 in upper bound represents "infinity" 
flexible_shape_utils.set_multiarray_ndshape_range(spec, 
                                 feature_name=input_name, 
                                 lower_bounds=[1,800,800,3], 
                                 upper_bounds=[-1,-1,-1,3])


In [254]:
mod = ct.models.MLModel(spec)

In [256]:
mod.save("retinaface_800-tf2-dynamic_shape.mlmodel")

In [174]:
# tf.keras.backend.set_learning_phase(0)
# model_pth = "./temp_with_weights_retinaface_800_ckpnt78.h5"
# model = tf.keras.models.load_model(model_pth,
#                                    custom_objects={"FPN": FPN, "SSH":SSH, "ClassHead": ClassHead, "BboxHead": BboxHead}, compile=False)


In [17]:
# model.summary()

In [ ]:
# import visualkeras

In [ ]:
# from PIL import ImageFont

# # font = ImageFont.truetype("arial.ttf", 32)  # using comic sans is strictly prohibited!
# visualkeras.layered_view(model, legend=True)

# try converting to dynamic shape

In [ ]:
import coremltools as ct


# import coremltools as ct
pipeline = ct.PassPipeline()
pipeline.remove_passes({"common::fuse_conv_batchnorm"})


print(ct.__version__)

input_name = model.inputs[0].name

height = 256
width = 256

input_shape = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
                    ct.RangeDim(lower_bound=height, upper_bound=-1),
                    ct.RangeDim(lower_bound=width, upper_bound=-1),
                    3))

# c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], pass_pipeline=pipeline, source='tensorflow')


# input_shape = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
#                     800,
#                     800,
#                     3))

# c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], pass_pipeline=pipeline, source='tensorflow')
c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], source='tensorflow')



# divide model into parts and try to convert 

In [25]:
# extractor = MobileNetV2(
#                 input_shape=None, include_top=False, weights="imagenet")

# import coremltools as ct


# # import coremltools as ct
# pipeline = ct.PassPipeline()
# pipeline.remove_passes({"common::fuse_conv_batchnorm"})


# print(ct.__version__)

# input_name = extractor.inputs[0].name

# height = 256
# width = 256

# input_shape = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
#                     ct.RangeDim(lower_bound=height, upper_bound=-1),
#                     ct.RangeDim(lower_bound=width, upper_bound=-1),
#                     3))

# # c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], pass_pipeline=pipeline, source='tensorflow')


# # input_shape = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
# #                     800,
# #                     800,
# #                     3))

# # c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], pass_pipeline=pipeline, source='tensorflow')
# c_model = ct.convert(extractor, inputs=[ct.TensorType(shape=input_shape, name=input_name)], source='tensorflow')

input_size = None

x = inputs = Input([input_size, input_size, 3], name='input_image')
preprocess = tf.keras.applications.mobilenet_v2.preprocess_input

extractor = MobileNetV2(
    input_shape= None, include_top=False, weights="imagenet")
pick_layer1 = 54  # [80, 80, 32]
pick_layer2 = 116  # [40, 40, 96]
pick_layer3 = 143  # [20, 20, 160]
preprocess = tf.keras.applications.mobilenet_v2.preprocess_input
extractor = Model(extractor.input,
                     (extractor.layers[pick_layer1].output,
                      extractor.layers[pick_layer2].output,
                      extractor.layers[pick_layer3].output),
                     name="mobilenet" + '_extrator')(preprocess(x))



extractor1 = Model(x, extractor)



In [26]:

extractor = MobileNetV2(
    input_shape=None, include_top=False, weights="imagenet")
pick_layer1 = 54  # [80, 80, 32]
pick_layer2 = 116  # [40, 40, 96]
pick_layer3 = 143  # [20, 20, 160]
preprocess = tf.keras.applications.mobilenet_v2.preprocess_input

(extractor.layers[54].name,
extractor.layers[pick_layer2].name,
extractor.layers[pick_layer3].name)


('block_6_expand', 'block_13_expand', 'block_16_expand')

In [27]:

(extractor.layers[54],
extractor.layers[pick_layer2],
extractor.layers[pick_layer3])


(<tensorflow.python.keras.layers.convolutional.Conv2D at 0x7f9dd82c45b0>,
 <tensorflow.python.keras.layers.convolutional.Conv2D at 0x7f9dd821cd90>)

In [28]:

cnt = 0

for layers_og, layers in zip(model.layers, extractor1.layers):
    if cnt == 3:
        print(layers_og, layers)
        for ly_og, ly in zip(layers_og.layers, layers.layers):
            ly.set_weights(ly_og.get_weights())
        break
    else:
        cnt += 1
    

<tensorflow.python.keras.engine.functional.Functional object at 0x7f9fd8173490> <tensorflow.python.keras.engine.functional.Functional object at 0x7f9df4162280>


In [29]:
import coremltools as ct

In [30]:

input_name = extractor1.inputs[0].name

height = 256
width = 256

input_shape = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
                    ct.RangeDim(lower_bound=height, upper_bound=-1),
                    ct.RangeDim(lower_bound=width, upper_bound=-1),
                    3))

# c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], pass_pipeline=pipeline, source='tensorflow')


# input_shape = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
#                     800,
#                     800,
#                     3))

# c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], pass_pipeline=pipeline, source='tensorflow')
c_model = ct.convert(extractor1, inputs=[ct.TensorType(shape=input_shape, name=input_name)], source='tensorflow')


2023-07-13 06:08:51.425101: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-07-13 06:08:51.425223: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-07-13 06:08:51.433838: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2023-07-13 06:08:51.921459: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-07-13 06:08:51.921608: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-07-13 06:08:51.989304: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 396 nodes (-241), 1128 ed

In [32]:
# full_model

In [33]:
extractor1.output

(<KerasTensor: shape=(None, None, None, 192) dtype=float32 (created by layer 'mobilenet_extrator')>,
 <KerasTensor: shape=(None, None, None, 576) dtype=float32 (created by layer 'mobilenet_extrator')>,
 <KerasTensor: shape=(None, None, None, 960) dtype=float32 (created by layer 'mobilenet_extrator')>)

In [34]:
model.layers[4].input

(<KerasTensor: shape=(None, None, None, 192) dtype=float32 (created by layer 'MobileNetV2_extrator')>,
 <KerasTensor: shape=(None, None, None, 576) dtype=float32 (created by layer 'MobileNetV2_extrator')>,
 <KerasTensor: shape=(None, None, None, 960) dtype=float32 (created by layer 'MobileNetV2_extrator')>)

In [35]:
# Input(shape=[[None, None, None, 192], (None, None, None, 576), (None, None, None, 960)])

[Input(shape= (None, None, None, 192)), Input(shape= (None, None, None, 576)), Input(shape= (None, None, None, 960))]


[<KerasTensor: shape=(None, None, None, None, 192) dtype=float32 (created by layer 'input_4')>,
 <KerasTensor: shape=(None, None, None, None, 576) dtype=float32 (created by layer 'input_5')>,
 <KerasTensor: shape=(None, None, None, None, 960) dtype=float32 (created by layer 'input_6')>]

In [36]:
extractor1.outputs

[<KerasTensor: shape=(None, None, None, 192) dtype=float32 (created by layer 'mobilenet_extrator')>,
 <KerasTensor: shape=(None, None, None, 576) dtype=float32 (created by layer 'mobilenet_extrator')>,
 <KerasTensor: shape=(None, None, None, 960) dtype=float32 (created by layer 'mobilenet_extrator')>]

In [37]:
cfg['min_sizes'][0]

[16, 32]

In [38]:
wd = cfg['weights_decay']
out_ch = cfg['out_channel']
print(out_ch)


x = (Input(shape= (None, None, 192)), Input(shape= (None, None, 576)), Input(shape= (None, None, 960)))

# x = (Input(shape= (100, 100, 192)), Input(shape= (50, 50, 576)), Input(shape= (25, 25, 960)))


# name = "CU"
# output1 = ConvUnit(f=out_ch, k=1, s=1, wd=wd, act="relu", name=name+str(1))
# output1(x[0])

fpn = FPN(out_ch=out_ch, wd=wd)
# fpn(x)

fpn_mdl = Model(inputs=x, outputs=fpn(x))
fpn_mdl.summary()


64
x[0] :- (None, None, None, 192)
x[1] :- (None, None, None, 576)
x[2] :- (None, None, None, 960)
Running output1 conv layer
Output of output1 conv :- (None, None, None, 64) 

Running output2 conv layer
Output of output2 conv :- (None, None, None, 64) 

Running output3 conv layer
Output of output3 conv :- (None, None, None, 64) 

Running resize of output3 

None None
Resized output3 to :- Tensor("FPN/Shape_2:0", shape=(4,), dtype=int32) (None, None, None, 64)

Adding output2 to resized output3 

Output2 shape :- Tensor("FPN/Shape_3:0", shape=(4,), dtype=int32) (None, None, None, 64)

passing output2 to merge2 conv 

Output of output2 :- (None, None, None, 64) 

Running resize of output2 w.r.t. output1 

Resized output2 to :- Tensor("FPN/Shape_6:0", shape=(4,), dtype=int32) (None, None, None, 64)

Adding output1 to resized output2 

Output1 shape :- Tensor("FPN/Shape_7:0", shape=(4,), dtype=int32) (None, None, None, 64)

Running merge1 conv on output1 

Shape of output1 :- (None, None,

In [39]:
fpn_mdl.inputs[2].name

'input_9'

In [40]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: fpn_mdl(x))
full_model = full_model.get_concrete_function(
    [
        tf.TensorSpec(fpn_mdl.inputs[0].shape, fpn_mdl.inputs[0].dtype), tf.TensorSpec(fpn_mdl.inputs[1].shape, fpn_mdl.inputs[1].dtype), 
        tf.TensorSpec(fpn_mdl.inputs[2].shape, fpn_mdl.inputs[2].dtype)
                   ])
     

x[0] :- (None, None, None, 192)
x[1] :- (None, None, None, 576)
x[2] :- (None, None, None, 960)
Running output1 conv layer
Output of output1 conv :- (None, None, None, 64) 

Running output2 conv layer
Output of output2 conv :- (None, None, None, 64) 

Running output3 conv layer
Output of output3 conv :- (None, None, None, 64) 

Running resize of output3 

None None
Resized output3 to :- Tensor("model_1/FPN/Shape_2:0", shape=(4,), dtype=int32) (None, None, None, 64)

Adding output2 to resized output3 

Output2 shape :- Tensor("model_1/FPN/Shape_3:0", shape=(4,), dtype=int32) (None, None, None, 64)

passing output2 to merge2 conv 

Output of output2 :- (None, None, None, 64) 

Running resize of output2 w.r.t. output1 

Resized output2 to :- Tensor("model_1/FPN/Shape_6:0", shape=(4,), dtype=int32) (None, None, None, 64)

Adding output1 to resized output2 

Output1 shape :- Tensor("model_1/FPN/Shape_7:0", shape=(4,), dtype=int32) (None, None, None, 64)

Running merge1 conv on output1 

Sha

In [41]:
# tf.compat.v1.global_variables_initializer().run()
# tf.initializers.global_variables_initializer().run()

frozen_func = convert_variables_to_constants_v2(full_model)
gph = frozen_func.graph.as_graph_def()

2023-07-13 06:09:12.939260: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-07-13 06:09:12.939380: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-07-13 06:09:12.942277: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.01ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



In [42]:
gph_nd = list(gph.node)

In [43]:
gph_nd[61]

name: "model_1/FPN/FPN1/bn/ReadVariableOp_1"
op: "Identity"
input: "model_1/FPN/FPN1/bn/ReadVariableOp_1/resource"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}

In [44]:


input_name = fpn_mdl.inputs[0].name
input_name1 = fpn_mdl.inputs[1].name
input_name2 = fpn_mdl.inputs[2].name

height = 200
width = 200


input_shape1 = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
                    ct.RangeDim(lower_bound=height, upper_bound=-1),
                    ct.RangeDim(lower_bound=width, upper_bound=-1),
                    192))

input_shape2 = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
                    ct.RangeDim(lower_bound=height, upper_bound=-1),
                    ct.RangeDim(lower_bound=width, upper_bound=-1),
                    576))

input_shape3 = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
                    ct.RangeDim(lower_bound=height, upper_bound=-1),
                    ct.RangeDim(lower_bound=width, upper_bound=-1),
                    960))




# c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], pass_pipeline=pipeline, source='tensorflow')


# input_shape = ct.Shape(shape=(ct.RangeDim(lower_bound=1, upper_bound=-1),
#                     800,
#                     800,
#                     3))

# c_model = ct.convert(model, inputs=[ct.TensorType(shape=input_shape, name=input_name)], pass_pipeline=pipeline, source='tensorflow')
fpn_model = ct.convert(fpn_mdl, inputs=[ct.TensorType(shape=input_shape1, name=input_name), ct.TensorType(shape=input_shape2, name=input_name1), ct.TensorType(shape=input_shape3, name=input_name2)], source='tensorflow')


# # out = fpn

# features = [SSH(out_ch=out_ch, wd=wd, name=f'SSH_{i}')(f)
#             for i, f in enumerate(fpn)]

# bbox_regressions = tf.concat(
#     [BboxHead(num_anchor, wd=wd, name=f'BboxHead_{i}')(f)
#      for i, f in enumerate(features)], axis=1)



# # landm_regressions = tf.concat(
# #     [LandmarkHead(num_anchor, wd=wd, name=f'LandmarkHead_{i}')(f)
# #      for i, f in enumerate(features)], axis=1)
# classifications = tf.concat(
#     [ClassHead(num_anchor, wd=wd, name=f'ClassHead_{i}')(f)
#      for i, f in enumerate(features)], axis=1)

# classifications = tf.keras.layers.Softmax(axis=-1)(classifications)


# bbox_reg = [BboxHead(num_anchor, wd=wd, name=f'BboxHead_{i}')(f)
#      for i, f in enumerate(features)]


# clf = [ClassHead(num_anchor, wd=wd, name=f'ClassHead_{i}')(f)
#      for i, f in enumerate(features)]

# out = (bbox_regressions, 
#        # landm_regressions, 
#        classifications)

# model_after_extractor = Model(fpn, out)

x[0] :- (None, None, None, 192)
x[1] :- (None, None, None, 576)
x[2] :- (None, None, None, 960)
Running output1 conv layer
Output of output1 conv :- (None, None, None, 64) 

Running output2 conv layer
Output of output2 conv :- (None, None, None, 64) 

Running output3 conv layer
Output of output3 conv :- (None, None, None, 64) 

Running resize of output3 

None None
Resized output3 to :- Tensor("model_1/FPN/Shape_2:0", shape=(4,), dtype=int32) (None, None, None, 64)

Adding output2 to resized output3 

Output2 shape :- Tensor("model_1/FPN/Shape_3:0", shape=(4,), dtype=int32) (None, None, None, 64)

passing output2 to merge2 conv 

Output of output2 :- (None, None, None, 64) 

Running resize of output2 w.r.t. output1 

Resized output2 to :- Tensor("model_1/FPN/Shape_6:0", shape=(4,), dtype=int32) (None, None, None, 64)

Adding output1 to resized output2 

Output1 shape :- Tensor("model_1/FPN/Shape_7:0", shape=(4,), dtype=int32) (None, None, None, 64)

Running merge1 conv on output1 

Sha

2023-07-13 06:09:16.673913: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-07-13 06:09:16.674050: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-07-13 06:09:16.676813: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2023-07-13 06:09:16.752773: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-07-13 06:09:16.752891: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-07-13 06:09:16.763719: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 72 nodes (-25), 150 edges

AttributeError: 'concat' object has no attribute 'y'

In [ ]:
extractor1.outputs

In [ ]:
# fpn(extractor1.outputs)

In [114]:
# extractor1.summary()